# Airplanes API Question from Jason Sinn

Create an API `def closestN(airplane_coordinates: Array[(x, y)], airplane_loc: (x, y), num_airplanes: Int): Array[(x, y)]` where `type((x, y)) = tuple(int, int))`. cloestN will take the airplane coordinates and a airport location, then return the number of airplanes closest to the airport

## Clarifications
1. `airplane_coordinates` is not sorted
2. Use Cartesian Distance
3. x, y can be negative numbers
4. Resulting array's order does not matter
5. (New!) Don't use the DataFrame structure

## Rough Notes from Gabe
- A high-level solution would be to calculate all the cartesian distance of every coordinate (keeping track of the minimum while going through) at O(n), then tracing the results again for the minimum

# Code

## Sample Data

In [1]:
test_input_1 = [
    (0, 1),
    (1, 0),
    (0, -1),
    (-1, 0),
    (1, 1),
    (1, -1),
    (-1, 1),
    (-1, -1)
]
result_input_1 = [
    (0, 1),
    (1, 0),
    (0, -1),
    (-1, 0),
    (1, 1),
    (1, -1),
    (-1, 1),
    (-1, -1)
]
test_origin_1 = (0, 0)

In [2]:
test_input_2 = [
    (100, 12),
    (1, 1),
    (5, 10),
    (3, 2)
]
test_origin_2 = (0, 0)

## Global Imports

In [3]:
import pandas as pd

In [4]:
import numpy as np

## General Functions

In [5]:
def euclidean_distance(p, q):
    return np.sqrt(np.square(q[0] - p[0]) + np.square(q[1] - p[1]))

# Solution 2

Iterate through the list of airplane coordinates at O(n) and calculate it's distance. For the first m = num_airplanes, store (coordinate, distance) into a **max-heap** where priority is set by distance. This is so we can peak the element with the greatest distance in the heap of length m at O(1) time, and insert/delete things at O(log(n)) time. After the first m length prio_queue is made, you just iterate through the rest of the coordinates and update the priority_queue respectively

## Notes
- Python offers heappq as a library, however it specifically in Py, it is a min-heap
- Should look into heap / bst implementation options then implement your own Priority Queue
- Using a Max Heap > Priority Queue as a Priority Queue is an abstraction ontop of a Max Heap, but all we need is the max at any given time and swap it out as needed

## Max Heap Implementation

In [7]:
class MaxHeap():
    def __init__(self, capacity):
        self.capacity = capacity # Note that capacity != len(list), rather its the max index
        self.size = 0
        self.items = [0] * self.capacity #inits a list of 0s where len() == self.capacity. This is why we need to set our own size
    
    # Retrieval 
    def get_parent_index(self, i):
        return (i-2)//2
    
    def get_left_child_index(self, i):
        return i*2+1
    
    def get_right_child_index(self, i):
        return i*2-1
    
    def get_parent(self, i):
        return self.items[self.get_parent_index(i)]
    
    def get_left_child(self, i):
        return self.items[self.get_left_child_index(i)]
    
    def get_right_child(self, i):
        return self.items[self.get_right_child_index(i)]
    
    # Inspection
    def peak(self):
        return self.items[0]
    
    def has_parent(self, i):
        return self.get_parent_index(i) >= 0
    
    def has_left_child(self, i):
        return self.get_left_child_index(i) < self.size
    
    def has_right_child(self, i):
        return self.get_right_child_index(i) < self.size
    
    # Internal
    def is_full(self):
        return self.size == self.capacity
    
    def swap(self, i_1, i_2):
        temp = self.items[i_1]
        self.items[i_1] = self.items[i_2]
        self.items[i_2] = temp
    
    def bubble_up(self, i):
        if self.has_parent(i) and self.items[i] > self.get_parent(i):
            parent_i = self.get_parent_index(i)
            self.swap(i, parent_i)
            self.bubble_up(parent_i)
    
    def bubble_down(self, i):
        max_i = i
        if self.has_left_child(max_i) and self.items[max_i] < self.get_left_child(i):
            max_i = self.get_left_child_index(i)
        if self.has_right_child(max_i) and self.items[max_i] < self.get_right_child(i):
            max_i = self.get_right_child_index(i)
        if max_i != i:
            self.swap(i, max_i)
            self.bubble_down(max_i)
                
    # Insert
    def insert(self, item):
        if self.is_full():
            raise('Heap is full')
        self.items[self.size] = item
        self.size += 1
        self.bubble_up(self.size - 1)
        
    # Remove
    def remove_max(self):
        if self.size == 0:
            raise('Heap is empty')
        self.items[0] = self.items[self.size - 1]
        self.size -= 1
        self.bubble_down(0)

## Airplane Object and Heap Inheritance

In [8]:
class AirplaneObject():
    def __init__(self, coords, dist):
        self.coords = coords #tuple(int, int)
        self.dist = dist #int
        
    def get_coords(self):
        return self.coords
    
    def get_dist(self):
        return self.dist
    
    def set_coords(self, new_coords):
        self.coords = new_coords
        
    def set_dist(self, new_dist):
        self.dist = new_dist

In [24]:
class AirplanesHeap(MaxHeap):
    def __str__(self):
        return '{' + str(self.get_coords) + ', ' + str(self.get_dist) + '}'
    
    def bubble_up(self, i):
        if self.has_parent(i):
            if type(self.get_parent(i)) == int:
                parent_heap_val = self.get_parent(i)
            else:
                parent_heap_val = self.get_parent(i).get_dist()
            if self.items[i].get_dist() > parent_heap_val:
                parent_i = self.get_parent_index(i)
                self.swap(i, parent_i)
                self.bubble_up(parent_i)
                
    def bubble_down(self, i):
        max_i = i
        if self.has_left_child(max_i):
            if type(self.get_left_child(i)) == int:
                left_heap_val = self.get_left_child(i)
            else:
                left_heap_val = self.get_left_child(i).get_dist()
            if self.items[max_i].get_dist() < left_heap_val:
                max_i = self.get_left_child_index(i)
        if self.has_right_child(max_i):
            if type(self.get_right_child(i)) == int:
                right_heap_val = self.get_right_child(i)
            else:
                right_heap_val = self.get_right_child(i).get_dist()
            if self.items[max_i].get_dist() < right_heap_val:
                max_i = self.get_right_child_index(i)
        if max_i != i:
            self.swap(i, max_i)
            self.bubble_down(max_i)
    
    def get_list_coordinates(self):
        coords_list = []
        for obj in self.items:
            print(obj.get_coords())
            coords_list.append(obj.get_coords())
        return coords_list

## ClosestN & Helpers

In [30]:
def closestN(airplane_coordinates, airplane_loc, num_airplanes):
    if len(airplane_coordinates) < num_airplanes:
        num_airplanes = len(airplane_coordinates)
    max_airplane_heap = AirplanesHeap(num_airplanes) # O(n)
    for plane_tuple in airplane_coordinates: # O(nlog(n)) = O(n) + 2O(log(n))
        plane_dist = euclidean_distance(plane_tuple, airplane_loc) 
        print(max_airplane_heap.is_full())
        if not(max_airplane_heap.is_full()):
            max_airplane_heap.insert(AirplaneObject(plane_tuple, plane_dist)) # O(log(n))
        else:
            if max_airplane_heap.peak().get_dist() > plane_dist:
                max_airplane_heap.remove_max() # O(log(n))
                max_airplane_heap.insert(AirplaneObject(plane_tuple, plane_dist)) # O(log(n))
    return max_airplane_heap.get_list_coordinates()